# Extract Simons Observatory instrument parameters

In order to be independent of `sotodlib` and track versions of the instrument model parameters more easily, we extract them to a file included in this repository.

In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
import sotodlib.sim_hardware as hardware

hw = hardware.sim_nominal()

In [4]:
import sotodlib

In [5]:
sotodlib.__version__

'0.2.1+3270.ga03ad3b3'

There is something wrong with versioning, anyway, the commit hash is:

```a03ad3b39d1625b2f50b8c5e231410ef770e7470```

In [6]:
fwhm_arcmin = {
    "SA": hw.data["telescopes"]["SAT1"]["fwhm"],
    "LA": hw.data["telescopes"]["LAT"]["fwhm"],
}

In [7]:
from pathlib import Path

In [8]:
from collections import OrderedDict
from astropy import units as u
from astropy.table import QTable

In [9]:
hw.data["bands"].keys()

odict_keys(['LAT_f030', 'LAT_f040', 'LAT_f090', 'LAT_f150', 'LAT_f230', 'LAT_f290', 'SAT_f030', 'SAT_f040', 'SAT_f090', 'SAT_f150', 'SAT_f230', 'SAT_f290', 'NC'])

In [10]:
fwhm_arcmin

{'SA': OrderedDict([('SAT_f030', 91.0),
              ('SAT_f040', 63.0),
              ('SAT_f090', 30.0),
              ('SAT_f150', 17.0),
              ('SAT_f230', 11.0),
              ('SAT_f290', 9.0)]),
 'LA': OrderedDict([('LAT_f030', 7.4),
              ('LAT_f040', 5.1),
              ('LAT_f090', 2.2),
              ('LAT_f150', 1.4),
              ('LAT_f230', 1.0),
              ('LAT_f290', 0.9)])}

In [11]:
channels = []
for band_label, band_properties in hw.data["bands"].items():
    if band_label == "NC":
        continue  # skip dark bolometers
    telescope = band_label[:3]
    print(band_label)
    ch = OrderedDict()
    ch["telescope"] = telescope
    ch["band"] = band_label
    print(ch["band"])
    ch["fwhm"] = fwhm_arcmin[telescope[:2]][band_label] * u.arcmin
    properties = hw.data["bands"][band_label]
    for freq_type in ["center", "low", "high"]:
        ch[freq_type + "_frequency"] = band_properties[freq_type] * u.GHz
    channels.append(ch)

LAT_f030
LAT_f030
LAT_f040
LAT_f040
LAT_f090
LAT_f090
LAT_f150
LAT_f150
LAT_f230
LAT_f230
LAT_f290
LAT_f290
SAT_f030
SAT_f030
SAT_f040
SAT_f040
SAT_f090
SAT_f090
SAT_f150
SAT_f150
SAT_f230
SAT_f230
SAT_f290
SAT_f290


## Read CSV of SO resolution and add to instrument model

In [12]:
from astropy.table import Table

In [13]:
so_default_resol = Table.read("so_default_resolution.csv")

In [14]:
so_default_resol.columns

<TableColumns names=('channel','fwhm','Nside','CAR_resol')>

In [15]:
so_default_resol.add_index("channel")

In [16]:
so_default_resol

channel,fwhm,Nside,CAR_resol
str8,float64,int64,float64
SAT_f030,91.0,128,12.0
SAT_f040,63.0,128,12.0
SAT_f090,30.0,512,4.0
SAT_f150,17.0,512,4.0
SAT_f230,11.0,1024,4.0
SAT_f290,9.0,1024,4.0
LAT_f030,7.4,2048,2.0
LAT_f040,5.1,2048,2.0
LAT_f090,2.2,8192,0.5


In [17]:
for ch in channels:
    ch_resol = so_default_resol.loc[ch["band"]]
    ch["nside"] = ch_resol["Nside"]
    ch["car_resol"] = ch_resol["CAR_resol"] * u.arcmin

## Add reference to bandpasses

In [18]:
[ch["band"][4:] for ch in channels[:6]]
bandname = {
    "f030": "LF1",
    "f040": "LF2",
    "f090": "MF1",
    "f150": "MF2",
    "f230": "HF1",
    "f290": "HF2",
}

In [19]:
from copy import deepcopy

In [20]:
wafers = []
n_wafers = 0
for ch in channels:
    bandpass_files = sorted(
        (Path("bandpass_sampler") / ch["telescope"]).glob(
            f"*{bandname[ch['band'][4:]]}*.tbl"
        )
    )
    print(ch["band"])
    for bandpass_file in bandpass_files:
        temp_ch = deepcopy(ch)
        print(bandpass_file)
        temp_ch["bandpass_file"] = str(bandpass_file)
        temp_ch["band"] = ch["band"] + "_" + bandpass_file.stem.split("_")[-1]
        wafers.append(temp_ch)
        n_wafers += 1

LAT_f030
bandpass_sampler/LAT/resampled_bpasses_LF1_w0.tbl
bandpass_sampler/LAT/resampled_bpasses_LF1_w1.tbl
bandpass_sampler/LAT/resampled_bpasses_LF1_w2.tbl
bandpass_sampler/LAT/resampled_bpasses_LF1_w3.tbl
LAT_f040
bandpass_sampler/LAT/resampled_bpasses_LF2_w0.tbl
bandpass_sampler/LAT/resampled_bpasses_LF2_w1.tbl
bandpass_sampler/LAT/resampled_bpasses_LF2_w2.tbl
bandpass_sampler/LAT/resampled_bpasses_LF2_w3.tbl
LAT_f090
bandpass_sampler/LAT/resampled_bpasses_MF1_w0.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w1.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w2.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w3.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w4.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w5.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w6.tbl
bandpass_sampler/LAT/resampled_bpasses_MF1_w7.tbl
LAT_f150
bandpass_sampler/LAT/resampled_bpasses_MF2_w0.tbl
bandpass_sampler/LAT/resampled_bpasses_MF2_w1.tbl
bandpass_sampler/LAT/resampled_bpasses_MF2_w2.tbl
bandpass_sampl

In [21]:
n_wafers

72

In [22]:
instrument_parameters = QTable(wafers)

In [23]:
ipac_warning = [
    "Text file in IPAC table format, read with astropy",
    "from astropy.table import QTable",
    "QTable.read('filename.tbl', format='ascii.ipac')",
]

In [24]:
instrument_parameters.meta["comments"] = [
    "sotodlib_version:" + sotodlib.__version__
] + ipac_warning

In [25]:
instrument_parameters

telescope,band,fwhm,center_frequency,low_frequency,high_frequency,nside,car_resol,bandpass_file
,,arcmin,GHz,GHz,GHz,,arcmin,
str3,str11,float64,float64,float64,float64,int64,float64,str49
LAT,LAT_f030_w0,7.4,25.7,21.7,29.7,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF1_w0.tbl
LAT,LAT_f030_w1,7.4,25.7,21.7,29.7,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF1_w1.tbl
LAT,LAT_f030_w2,7.4,25.7,21.7,29.7,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF1_w2.tbl
LAT,LAT_f030_w3,7.4,25.7,21.7,29.7,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF1_w3.tbl
LAT,LAT_f040_w0,5.1,38.9,30.9,46.9,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF2_w0.tbl
LAT,LAT_f040_w1,5.1,38.9,30.9,46.9,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF2_w1.tbl
LAT,LAT_f040_w2,5.1,38.9,30.9,46.9,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF2_w2.tbl
LAT,LAT_f040_w3,5.1,38.9,30.9,46.9,2048,2.0,bandpass_sampler/LAT/resampled_bpasses_LF2_w3.tbl


In [26]:
instrument_parameters.write(
    "instrument_parameters.tbl", format="ascii.ipac", overwrite=True
)